In [1]:
#Import Necessary Packages
import requests
import pandas as pd

In [2]:
#Pull all Characters into Df_Charactersrm
df_Charactersrm = pd.DataFrame()
for page in range(1,35):
    url='https://rickandmortyapi.com/api/character/?page=%s'%page
    response = requests.get(url)
    json_string = response.json()
    dftempchar = pd.json_normalize(json_string, record_path =['results'])
    df_Charactersrm = pd.concat([df_Charactersrm, dftempchar])
    


In [3]:
#Pull all Locations into Df_Locationsrm
df_Locationsrm = pd.DataFrame()
for page in range(1,7):
    url='https://rickandmortyapi.com/api/location/?page=%s'%page
    response = requests.get(url)
    json_string = response.json()
    dftemploc = pd.json_normalize(json_string, record_path =['results'])
    df_Locationsrm = pd.concat([df_Locationsrm, dftemploc])

In [4]:
#Pull all Episodes into Df_Episodesrm
df_Episodesrm = pd.DataFrame()
for page in range(1,4):
    url='https://rickandmortyapi.com/api/episode/?page=%s'%page
    response = requests.get(url)
    json_string = response.json()
    dftempep = pd.json_normalize(json_string, record_path =['results'])
    df_Episodesrm = pd.concat([df_Episodesrm, dftempep])

In [5]:
#Confirm Everything is There
print(df_Charactersrm.shape)
print(df_Locationsrm.shape)
print(df_Episodesrm.shape)

(671, 14)
(108, 7)
(41, 7)


In [6]:
#Now we will want to duplicate rows in the df_Charactersrm so that we have 1 row for each episode the character appeared in
df_Charactersrm =df_Charactersrm.explode('episode').reset_index(drop=True)
cols = list(df_Charactersrm.columns)
cols.append(cols.pop(cols.index('name')))
df_Charactersrm =df_Charactersrm[cols]

In [7]:
df_Charactersrm.shape

(1038, 14)

In [8]:
#Lets Do Some DF Cleanup pre Join

#Drop Unecessary Columns
df_Charactersrm = df_Charactersrm.drop(columns=['created','location.url','url','origin.url'])
df_Locationsrm = df_Locationsrm.drop(columns=['id','created','residents','url'])
df_Episodesrm = df_Episodesrm.drop(columns=['id','created','characters'])

#Rename Columns
df_Charactersrm = df_Charactersrm.rename(columns={"status":"Character_Status","species":"Character_Species","type":"Character_type","image":"Character_image","episode":"Episode_URL","origin.name":"Origin_Location_Name","location.name":"Current_Location_Name","name":"Character_Name"})
df_Episodesrm = df_Episodesrm.rename(columns={"url":"Episode_URL","name":"episode_name","episode":"episode_Number"})
#Rename Columns and Make 2 dfs for Location one for each join
df_Locationsrm_Current = df_Locationsrm.rename(columns={"name":"Current_Location_Name","type":"Current_Location_type","dimension":"Current Dimension"})
df_Locationsrm_Origin = df_Locationsrm.rename(columns={"name":"Origin_Location_Name","type":"Origin_Location_type","dimension":"Origin Dimension"})


In [9]:
#Join df_Charactersrm with df_Episodesrm to get episode information for each character
df_rm = df_Charactersrm.merge(df_Episodesrm, on='Episode_URL', how='left')
print(df_rm.shape)
#Join df_rm with df_Locationsrm_Current to get current location information
df_rm = df_rm.merge(df_Locationsrm_Current, on='Current_Location_Name', how='left')
print(df_rm.shape)
#Join df_rm with df_Locationsrm_Origin to get Origin location information
df_rm = df_rm.merge(df_Locationsrm_Origin, on='Origin_Location_Name', how='left')
print(df_rm.shape)

(1038, 13)
(1038, 15)
(1038, 17)


In [10]:
df_rm =df_rm.drop(columns=['Episode_URL'])

In [11]:
df_Charactersrm.shape

(1038, 10)

In [12]:
df_rm.to_csv('Rick_And_Morty_Combined.csv')

In [13]:
df_rm.shape

(1038, 16)